In [35]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math

In [36]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-13,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-13,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-13,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-13,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-13,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [37]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7803


In [38]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 79
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [39]:
print(screener_df.head())
print()
print(len(screener_df.index))

  symbol     marketcap exchange                 exchangeName
0      A  3.830615e+10     XNYS  New York Stock Exchange Inc
1     AA  4.658117e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.019380e+07     XNAS           Nasdaq All Markets
3   AACI  2.576050e+07     XNAS           Nasdaq All Markets
4  AACIU  2.576050e+07     XNAS           Nasdaq All Markets

7803


In [40]:
# drop NAs
screener_df.dropna(inplace=True)

In [41]:
# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()
print(screener_df.head())

New length: 7700

  symbol     marketcap exchange                 exchangeName
0      A  3.830615e+10     XNYS  New York Stock Exchange Inc
1     AA  4.658117e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.019380e+07     XNAS           Nasdaq All Markets
3   AACI  2.576050e+07     XNAS           Nasdaq All Markets
4  AACIU  2.576050e+07     XNAS           Nasdaq All Markets


In [42]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

# apply function to 'marketcap' column AND create new 'marketcapType' column
screener_df['marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)
print(screener_df.head(10))

   symbol     marketcap exchange                 exchangeName marketcapType
0       A  3.830615e+10     XNYS  New York Stock Exchange Inc     Large-Cap
1      AA  4.658117e+09     XNYS  New York Stock Exchange Inc       Mid-Cap
2    AACG  4.019380e+07     XNAS           Nasdaq All Markets     Micro-Cap
3    AACI  2.576050e+07     XNAS           Nasdaq All Markets     Micro-Cap
4   AACIU  2.576050e+07     XNAS           Nasdaq All Markets     Micro-Cap
5    AACT  5.255000e+08     XNYS  New York Stock Exchange Inc     Small-Cap
7   AACT=  5.250000e+08     XNYS  New York Stock Exchange Inc     Small-Cap
8    AADI  4.390129e+07     XNAS           Nasdaq All Markets     Micro-Cap
9    AADR  2.563650e+07     XNAS           Nasdaq All Markets     Micro-Cap
10   AAGR  5.034414e+07     XNAS           Nasdaq All Markets     Micro-Cap


In [43]:
# DEV - marketcap groupby count
marketcap_counts = screener_df['marketcapType'].value_counts()

# convert SERIES to DF
marketcap_counts_df = marketcap_counts.reset_index()
marketcap_counts_df.columns = ['marketcapType', 'count']

# add percentage column
total_count = marketcap_counts.sum()
marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# print
marketcap_counts_df

,marketcapType,count,percentage
0,Micro-Cap,3523,45.753247
1,Small-Cap,1935,25.129870
2,Mid-Cap,1240,16.103896
3,Large-Cap,933,12.116883
4,Mega-Cap,69,0.896104


In [44]:
# DEV - checker
total_count_CHECK = marketcap_counts_df['count'].sum()

# if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
if total_count_CHECK == len(screener_df):
    print('Data lengths match! SUCCESS')
else:    
    print('Data mismatch! ERROR')
    print()
    print(f'Code checker: {total_count_CHECK}')
    print(f'Data length: {len(screener_df)}')

Data lengths match! SUCCESS
